<a href="https://colab.research.google.com/github/Aayushktyagi/Pytorch_resources/blob/master/Efficientnet_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Stanford Dog classification 

*   Loading dataset 
  1.   wget images
  2.   unzip
  3.   resize to 256x256


*   Dataset Visualization
*   Initialize EfficientNet
*   Train EfficientNet
*   Evaluate model 
*   Visualize predictions 







In [ ]:
import os
from os import path

In [ ]:
# load dataset
if os.path.exists("images.tar") and os.path.exists("annotation.tar"):
  # unzip images and annotations 
  print("file exist: Unzipping ....")
  !tar -xvf images.tar
  !tar -xvf annotation.tar
else:
  !wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
  !wget http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar
  # unzip images and annotations 
  !tar -xvf images.tar
  !tar -xvf annotation.tar

In [ ]:
import torch 
import numpy as np
from torchvision import models, datasets , transforms

In [ ]:


# folder_name = 'Images'

# images_datasets = {x: datasets.ImageFolder(folder_name,
#                                            data_transforms[x])
#                   for x in ['Train','Test']} 
# dataloader = {x: torch.utils.data.DataLoader(images_datasets[x],batch_size = 8,
#                                              shuffle = True , num_workers = 4)
#                   for x in ['Train', 'Test']}

# dataset_size = {x: len(images_datasets[x]) for x in ['Train','Test']}
# print(dataset_size)


{'Train': 20580, 'Test': 20580}


In [ ]:
# pre-process datasest

data_transforms = {
    'Train': transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(45),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 406],[0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])
}

class CustomDataSet(Dataset):
  def __init__(self, dataset, transform = None):
    self.dataset = dataset
    self.transform = transform

  def __getitem__(self, index):
    if self.transform:
      x = self.transform(self.dataset[index][0])
    else:
      x = self.dataset[index][0]
    y = self.dataset[index][1]

  def __len__(self):
    return len(self.dataset)

#get data instance
imagedatafolder = datasets.ImageFolder(folder_name)
traindataset = CustomDataSet(imagedatafolder , data_transform['Train'])
validdataset = CustomDataSet(imagedatafolder , data_transform['Test'])
testdataset = CustomDataSet(imagedatafolder, data_transform['Test'])

# train valid test split

train_size = 0.8
data_shuffle = True
random_seed = 42
batch_size = 32
num_worker = 4

dataset_size = len(imagedatafolder)
indices = list(range(dataset_size))

split = int(np.floor(train_size * dataset_size))
split2 = int(np.floor(train_size + ((1-train_size) / 2)) * dataset_size)

if data_shuffle:
  np.random.seed(random_seed)
  np.random.shuffle(indices)

train_idx , valid_idx , test_idx = indices[:split] , indices[split:split1] , indices[split2:]

trainsplit = Subset(traindataset , train_idx)
validsplit = Subset(validdataset , valid_idx)
testsplit = Subset(testdataset , test_idx)

trainloader = torch.utils.data.DataLoader(trainsplit , batch_size = batch_size,
                                          num_worker = num_worker , drop_last = True)
validloader = torch.utils.data.DataLoader(validsplit, batch_size = batch_size,
                                          num-worker = num_worker , drop_last= True)
testloader = torch.utils.data.DataLoader(testsplit, batch_size = batch_size,
                                         num_worker= num_worker, drop_last= True)

# check size of train, valid, test

print("Train:{} , Valid:{} , Test:{}".format(len(trainloader),len(validloader),len(testloader)))

